# __Dense matrix tests__

## Outline
- __Dependencies__
- __Dynamic ansatz__
    - Noiseless
    - With noise using Qiskit AER
- __Static ansatz__
    - Noiseless
    - With noise using Qiskit AER

In [1]:
import numpy as np
from device_var_lse_solver import DeviceVarLSESolver
from device import DeviceType
from conditioned_matrix import conditionedMatrix, poissonMatrix
from non_dynamical_ansatz import fixed_layered_ansatz
from tqdm import tqdm
from device import Device

qubits = 4
b = np.ones(2**qubits)/np.sqrt(2**qubits)
depth=10

assert qubits % 2 == 0

In [2]:
# As defined by [10]
def TRC_ADA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += d*z
    return trc

def TRC_ASA(iteration_count : dict):
    trc = 0
    for d in iteration_count:
        z = iteration_count.get(d)
        trc += z*depth
    return trc

## __Dynamic ansatz__

In [ ]:
# For poisson matrix
N_runs = 20#20
N_steps = 1000#100
max_c = 2#20
redo_calc = 1#10

cs = [2*i+1 for i in range(int(max_c/2))]

local_noiseless_dynamic_trc = np.zeros((redo_calc, len(cs)))
global_noiseless_dynamic_trc = np.zeros((redo_calc, len(cs)))
local_noise_dynamic_trc = np.zeros((redo_calc, len(cs)))
global_noise_dynamic_trc = np.zeros((redo_calc, len(cs)))

local_noiseless_static_trc = np.zeros((redo_calc, len(cs)))
global_noiseless_static_trc = np.zeros((redo_calc, len(cs)))
local_noise_static_trc = np.zeros((redo_calc, len(cs)))
global_noise_static_trc = np.zeros((redo_calc, len(cs)))

local_noise_dynamic_error = np.zeros((redo_calc, len(cs)))
local_noiseless_dynamic_error = np.zeros((redo_calc, len(cs)))
local_noise_static_error = np.zeros((redo_calc, len(cs)))
local_noiseless_static_error = np.zeros((redo_calc, len(cs)))

global_noise_dynamic_error = np.zeros((redo_calc, len(cs)))
global_noiseless_dynamic_error = np.zeros((redo_calc, len(cs)))
global_noise_static_error = np.zeros((redo_calc, len(cs)))
global_noiseless_static_error = np.zeros((redo_calc, len(cs)))

device = Device(DeviceType.QISKIT_AER, qubits=qubits)

param_shape = (qubits + depth*(qubits+qubits -2),)

for i in range(redo_calc):
        print(f"Run nr. {i+1}/{redo_calc}")
        for j, c in enumerate(cs):
                #a = conditionedMatrix(2**qubits, 1)
                #a = conditionedMatrix(2**qubits, 10)
                #a = conditionedMatrix(2**qubits, 100)
                a = poissonMatrix(2**(qubits//2)).todense()
                classical_solution = np.linalg.solve(a, b)
                normalized_classical_solution = np.square(classical_solution / np.linalg.norm(classical_solution))
                normalized_classical_solution /= np.linalg.norm(normalized_classical_solution)

                ###################################
                #####    DYNAMIC NOISEFULL    #####
                ###################################
                lse_local = DeviceVarLSESolver(a,
                        b, 
                        method='direct',
                        ansatz=fixed_layered_ansatz,
                        weights=param_shape,
                        local=True, 
                        lr=0.1, 
                        steps=N_steps,
                        threshold=0.001, 
                        epochs=N_runs,
                        device=device,
                        silent=False)
        
                solution_local, param_local, it_count_local = lse_local.solve()
                solution_local /= np.linalg.norm(solution_local)

                local_noise_dynamic_trc[i, j] = TRC_ADA(it_count_local)

                local_noise_dynamic_error[i, j] = np.linalg.norm(solution_local - np.abs(normalized_classical_solution))

                print(f"Solution local:\n{solution_local}\n")
                print(f"Classical solution:\n{np.abs(normalized_classical_solution)}\n")
print(f"Difference:\n{local_noise_dynamic_error}\n")



c:\QuantumAlgs-WI4650\QuantumAlgorithmsAssignments\venv\Lib\site-packages\variational_lse_solver\var_lse_solver.py:219: UserWarning: Explicit `ansatz` was provided, `epochs` argument will be ignored.
  warnings.warn('Explicit `ansatz` was provided, `epochs` argument will be ignored.')


Run nr. 1/1
Epoch 1/1:  47%|████▋     | 469/1000 [00:50<00:57,  9.20it/s, best loss=0.00121, last improvement in step=326, loss=0.00198]
Loss of 0.0007639852 below stopping threshold.
Returning solution.
Solution local:
[0.11759047 0.18173072 0.19598411 0.14609725 0.17460403 0.40265827
 0.29575784 0.2173642  0.26012437 0.35633475 0.45967183 0.22449089
 0.10690042 0.18885742 0.23518093 0.08552034]

Classical solution:
[0.10063969 0.19725378 0.19725378 0.10063969 0.19725378 0.40255874
 0.40255874 0.19725378 0.19725378 0.40255874 0.40255874 0.19725378
 0.10063969 0.19725378 0.19725378 0.10063969]

Difference:
[[0.16367673]]



In [4]:
solution_local = [0.31597247 , 0.2034898  , 0.33569425 , 0.26231688 , 0.21888156 , 0.16515955
 , 0.28294968 , 0.24501996 , 0.14914299 , 0.19434387 , 0.12252839 , 0.25144405
 , 0.20440255 , 0.25646057 , 0.27321977 , 0.37618937]

classical_solution = [0.24696282 , 0.23395656 , 0.38078167 , 0.2358157  , 0.1991396  , 0.15171504
 , 0.29446955 , 0.24106469 , 0.18049726 , 0.21884876 , 0.10432878 , 0.240708
 , 0.24202995 , 0.2201995  , 0.24094129 , 0.40218595]

solution_local /= np.linalg.norm(solution_local)
solution_local /= np.linalg.norm(solution_local)
print(solution_local)
classical_solution /= np.linalg.norm(classical_solution)
print(classical_solution)
classical_solution /= np.linalg.norm(classical_solution)

print(np.abs(solution_local-classical_solution))
a = sum(np.abs(solution_local-classical_solution)**2)
print(a)

print(np.linalg.norm(np.abs(solution_local-classical_solution))**2)

[0.31597247 0.2034898  0.33569425 0.26231688 0.21888156 0.16515955
 0.28294968 0.24501996 0.14914299 0.19434387 0.12252839 0.25144405
 0.20440255 0.25646057 0.27321977 0.37618937]
[0.24696282 0.23395656 0.38078167 0.2358157  0.1991396  0.15171504
 0.29446955 0.24106469 0.18049726 0.21884876 0.10432878 0.240708
 0.24202995 0.2201995  0.24094129 0.40218595]
[0.06900965 0.03046676 0.04508742 0.02650118 0.01974196 0.01344451
 0.01151987 0.00395527 0.03135427 0.02450489 0.01819961 0.01073605
 0.0376274  0.03626107 0.03227848 0.02599658]
0.015623072023670116
0.015623072023670113


In [5]:
print("Condition number 10:")
print(np.mean(local_noise_dynamic_error[0]))

Condition number 10:
0.23509580355432466


## __Save vals__

In [6]:
## Save the vals

## __Plot?__

In [7]:
amp = 0.05

# Dynamic noisy
local_dynamic_noise_color = "tab:blue"
plt.plot(cs, local_noise_dynamic_mean, color=local_dynamic_noise_color, label=f'Local dynamic noisy TRC expectation with {amp}*variance')
plt.fill_between(cs, local_noise_dynamic_mean-amp*local_noise_dynamic_variance, local_noise_dynamic_mean+amp*local_noise_dynamic_variance, color=local_dynamic_noise_color, alpha=0.4)

global_dynamic_noise_color = "tab:blue"
plt.plot(cs, global_noise_dynamic_mean, color=global_dynamic_noise_color, label=f'Global dynamic noisy TRC expectation with {amp}*variance')
plt.fill_between(cs, global_noise_dynamic_mean-amp*global_noise_dynamic_variance, global_noise_dynamic_mean+amp*global_noise_dynamic_variance, color=global_dynamic_noise_color, alpha=0.4)

# Dynamic noiseless
local_dynamic_noiseless_color = "tab:blue"
plt.plot(cs, local_noiseless_dynamic_mean, color=local_dynamic_noiseless_color, label=f'Local dynamic noiseless TRC expectation with {amp}*variance')
plt.fill_between(cs, local_noiseless_dynamic_mean-amp*local_noiseless_dynamic_variance, local_noiseless_dynamic_mean+amp*local_noiseless_dynamic_variance, color=local_dynamic_noiseless_color, alpha=0.4)

global_dynamic_noiseless_color = "tab:blue"
plt.plot(cs, global_noiseless_dynamic_mean, color=global_dynamic_noiseless_color, label=f'Global dynamic noiseless TRC expectation with {amp}*variance')
plt.fill_between(cs, global_noiseless_dynamic_mean-amp*global_noiseless_dynamic_variance, global_noiseless_dynamic_mean+amp*global_noiseless_dynamic_variance, color=global_dynamic_noiseless_color, alpha=0.4)

# Static noisy
local_static_noise_color = "tab:blue"
plt.plot(cs, local_noise_static_mean, color=local_static_noise_color, label=f'Local static noisy TRC expectation with {amp}*variance')
plt.fill_between(cs, local_noise_static_mean-amp*local_noise_static_variance, local_noise_static_mean+amp*local_noise_static_variance, color=local_static_noise_color, alpha=0.4)

global_static_noise_color = "tab:blue"
plt.plot(cs, global_noise_static_mean, color=global_static_noise_color, label=f'Global static noisy TRC expectation with {amp}*variance')
plt.fill_between(cs, global_noise_static_mean-amp*global_noise_static_variance, global_noise_static_mean+amp*global_noise_static_variance, color=global_static_noise_color, alpha=0.4)

# Static noiseless
local_static_noiseless_color = "tab:blue"
plt.plot(cs, local_noiseless_static_mean, color=local_static_noiseless_color, label=f'Local static noiseless TRC expectation with {amp}*variance')
plt.fill_between(cs, local_noiseless_static_mean-amp*local_noiseless_static_variance, local_noiseless_static_mean+amp*local_noiseless_static_variance, color=local_static_noiseless_color, alpha=0.4)

global_static_noiseless_color = "tab:blue"
plt.plot(cs, global_noiseless_static_mean, color=global_static_noiseless_color, label=f'Global static noiseless TRC expectation with {amp}*variance')
plt.fill_between(cs, global_noiseless_static_mean-amp*global_noiseless_static_variance, global_noiseless_static_mean+amp*global_noiseless_static_variance, color=global_static_noiseless_color, alpha=0.4)

#############################

plt.xlabel("Conditional number")
plt.ylabel("TRC score")

plt.legend()
plt.show()

NameError: name 'plt' is not defined